<a href="https://colab.research.google.com/github/ja8an/big-data/blob/main/TDE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
!wget -P -Q https://github.com/ja8an/big-data/blob/main/in/transactions_amostra.csv?raw=true -O transactions.csv
!pip install -q findspark
import findspark
findspark.init()
!pip install -q pyspark
from pyspark import SparkConf, SparkContext, SparkFiles

conf = SparkConf().setAppName("teste").setMaster("local[*]")
sc = SparkContext.getOrCreate(conf = conf)

--2022-10-24 09:17:56--  https://github.com/ja8an/big-data/blob/main/in/transactions_amostra.csv?raw=true
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/ja8an/big-data/raw/main/in/transactions_amostra.csv [following]
--2022-10-24 09:17:56--  https://github.com/ja8an/big-data/raw/main/in/transactions_amostra.csv
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://media.githubusercontent.com/media/ja8an/big-data/main/in/transactions_amostra.csv [following]
--2022-10-24 09:17:56--  https://media.githubusercontent.com/media/ja8an/big-data/main/in/transactions_amostra.csv
Resolving media.githubusercontent.com (media.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to media.githubusercontent.com (media.githubusercontent.com)|185.199.108.1

In [12]:
dataset = sc.textFile("/content/transactions.csv")
dataset = dataset.flatMap(lambda line : line.split('\n'))
dataset = dataset.filter(lambda line : line.strip() != 'country_or_area;year;comm_code;commodity;flow;trade_usd;weight_kg;quantity_name;quantity;category')
dataset = dataset.map(lambda cols : cols.split(';'))

In [13]:
def sum(a, b):
  if type(a) is tuple and type(b) is tuple:
    return (a[0] + b[0], a[1] + b[1])
  return a + b

(Easy) The number of transactions involving Brazil;

In [14]:
dataset.map(lambda cols : (cols[0], 1)).filter(lambda a : a[0] == 'Brazil').reduceByKey(sum).first()

('Brazil', 54762)

(Easy) The number of transactions per year;

In [15]:
dataset.map(lambda cols : (cols[1], 1)).reduceByKey(sum).take(5)

[('2011', 105486),
 ('1991', 22652),
 ('2005', 105913),
 ('1999', 86191),
 ('2000', 97572)]

(Easy) The number of transactions per flow type and year;

In [16]:
dataset.map(lambda cols: ((cols[4], cols[1]), 1)).reduceByKey(sum).take(5)

[(('Export', '2016'), 30538),
 (('Export', '1996'), 25472),
 (('Import', '2013'), 61273),
 (('Export', '2015'), 35759),
 (('Import', '1999'), 52260)]

(Easy) The average of commodity values per year;

In [17]:
_map = dataset.map(lambda cols: (cols[1], (float(cols[5]), 1)))
_sum = _map.reduceByKey(sum)
_mean = _sum.mapValues(lambda a: a[0] / a[1])
_mean.take(5)

[('2011', 37056771.709060915),
 ('1991', 12069165.192609925),
 ('2005', 17458172.20780263),
 ('1999', 9328194.404264946),
 ('2000', 12780250.522332227)]

(Easy) The average price of commodities per unit type, year, and category in the export flow in Brazil;

In [18]:
_filter = dataset.filter(lambda cols: cols[0] == 'Brazil' and cols[4] == 'Export')
_map = _filter.map(lambda cols: ((cols[7], cols[1], cols[9]), (float(cols[5]), 1)))
_sum = _map.reduceByKey(sum)
_mean = _sum.mapValues(lambda a: a[0] / a[1])
_mean.take(5)

[(('Weight in kilograms', '2009', '37_photographic_or_cinematographic_goods'),
  181305.5),
 (('Weight in kilograms', '2010', '52_cotton'), 1874796.0),
 (('Weight in kilograms', '2014', '18_cocoa_and_cocoa_preparations'),
  76505492.0),
 (('Weight in kilograms', '2016', '73_articles_of_iron_or_steel'),
  5815545.909090909),
 (('Number of items', '2002', '95_toys_games_sports_requisites'),
  120856.42857142857)]

(Medium) The maximum, minimum, and mean transaction price per unit type and year;

In [19]:
_map = dataset.map(lambda cols: ((cols[7], cols[1]), (float(cols[5]), 1, float(cols[5]), float(cols[5]))))
_sum = _map.reduceByKey(lambda a, b: (a[0] + b[0], a[1] + b[1], max(a[2], b[2]), min(a[3], b[3])))
_mean = _sum.mapValues(lambda a: {
    'mean': a[0] / a[1],
    'max': a[2],
    'min': a[3]
})
_mean.take(5)

[(('Weight in kilograms', '2016'),
  {'mean': 27962304.5069153, 'max': 56865611547.0, 'min': 1.0}),
 (('Weight in kilograms', '2010'),
  {'mean': 28822456.939314935, 'max': 105814257878.0, 'min': 1.0}),
 (('Weight in kilograms', '1990'),
  {'mean': 10720973.53031925, 'max': 4480755200.0, 'min': 1.0}),
 (('Weight in kilograms', '2014'),
  {'mean': 41934737.83217836, 'max': 359475936313.0, 'min': 1.0}),
 (('Weight in kilograms', '2006'),
  {'mean': 22895231.261773862, 'max': 262924347374.0, 'min': 1.0})]

(Hard) The most commercialized commodity (summing the quantities) in 2016, per flow type.

In [20]:
_filter = dataset.filter(lambda cols: cols[1] == '2016') # Only 2016
_map = _filter.map(lambda a: ((a[4], a[3]), float(a[8])))
_sum = _map.reduceByKey(sum)
_map = _sum.map(lambda a: (a[0][0], (a[0][1], a[1])))
_max = _map.reduceByKey(lambda a, b: a if a[1] > b[1] else b)
_map = _max.mapValues(lambda a: a[0])
_map.collect()

[('Re-Export', 'Safety razor blades, including blanks in strips'),
 ('Export', 'Ice, snow and potable water not sweetened or flavoure'),
 ('Re-Import', 'Chem wood pulp, soda/sulphate, non-conifer, bleached'),
 ('Import', 'Iron ore, concentrate, not iron pyrites,unagglomerate')]